In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import ssl
import os
import re

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [2]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
class infine_scroll(object): 
  def __init__(self, last):
    self.last = last

  def __call__(self, driver):
    new = driver.execute_script('return document.body.scrollHeight')  
    if new > self.last:
        return new
    else:
        return False

In [19]:


def get_html_from_url(url):
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver') 
  browser.set_page_load_timeout(30) 
  browser.get(url)
  
  html= '' 
  
  last_height = browser.execute_script('return document.body.scrollHeight')

  html += browser.page_source
  flag=1

  while flag==1:
    browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')
 
    html += browser.page_source
    try:
       wait = WebDriverWait(browser, 10)

       new_height = wait.until(infinite_scroll(last_height))
       last_height = new_height

    except:
        print("End of page reached")
        flag = 0
  return html

In [5]:
#helper function get first number in string

def str_first_num(s):
    first_num = ''
    flag_digit=False
    for el in s: 
        if el.isdigit():
            first_num += el
            flag_digit = True
        else:
            if flag_digit:
                break
    #If there are no upvotes replace '' with 0
    if first_num == '':
        first_num = 0
    return first_num
    

In [6]:
# a helper function that removes unwanted words/sections like url links
def strip_rgx_words(inp, regex_ = ['image','url']):
    #remove urls or images
    rgx_word = None
    for rgx in regex_:
      if rgx in inp:
          rgx_word = rgx
          break
    if rgx_word is not None:
      out = inp.split(rgx_word)[0]
    else:
      out =inp
    return out

In [7]:
def compare_two_str(st1, st2):
    '''
    input: two strings 
    output: differences between two strings and indexes'''
    unsimilar_id = {}
    for i, l1 in enumerate(st1):
        l2 = st2[i]
        if l1 != l2:
            unsimilar_id[i] = [l1, l2]
    return unsimilar_id

In [8]:
#read question urls from csv file
src_path = r"C:\Users\Ravit\Documents\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\quet_indexes.csv"
df = pd.read_csv(src_path)

In [9]:
#get list of urls
url_list = list(df['url'].values)

In [10]:
url_list

['https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/How-long-could-the-brain-survive-theoretically-if-we-had-the-technology-to-replace-all-other-body-parts-as-you-aged-with-functioning-organs-that-are-grown-in-a-lab-using-your-DNA',
 'https://www.quora.com/What-are-the-top-10-emerging-technologies-in-the-next-5-10-years-2020%E2%80%932025',
 'https://www.quora.com/What-are-the-upcoming-emerging-technologies-in-software-industry',
 'https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/What-are-the-most-advanced-technologies-that-people-dont-know-about-yet',
 'https://www.quora.com/When-will-Fusion-reactors-become-a-reality',
 'https://www.quora.com/In-future-IOT-Internet-of-things-is-trending-technology-or-not',
 'https://www.quora.com/What-will-the-worlds-technology-be-like-in-50-years',
 'https://www.quora.com/What-are-the-solutions-to-emerging-issues-in-communication']

In [11]:
def answers_into_df(url):
    '''input: url for scrapping answers to a particular question.
        out: df with answers for each url/question'''
    #request url html
    
    #create soup object with htmal parser
    page_html = get_html_from_url(url)
    soup = BeautifulSoup(page_html)

    answers = soup.find_all('p', {'class':"q-text qu-display--block qu-wordBreak--break-word qu-textAlign--start"})
    answers = list(set(answers))
    
    #get all answers in span
    answers = [ans.find('span') for ans in answers]
    answers = [ans.find_next(text=True).strip() for ans in answers]
    #remove duplicates 
    answers = list(set(answers))
    
    upvotes = soup.find_all('span', {'class':"q-text qu-whiteSpace--nowrap qu-display--inline-flex qu-alignItems--center qu-justifyContent--center"})
    upvotes = [up.find_next(text=True).strip() for up in upvotes]
    

    
    #convert soup object to string
    answers = [str(a) for a in answers][0]
    #split answers with text separator
    answers_list = answers.split('"text":')
    #first element is just the header of a page. remove it
    answers_list = answers_list[1:]
    
    #create df with nans to fill it later with values
    columns = ["Answer", "upvoteCount", "answerCount", "followerCount", "name"]
    #columns = ["Answer", "upvoteCount"]
    x_shape = (len(answers_list), len(columns))
    x = np.tile(np.nan, x_shape)
   
    answers_df = pd.DataFrame(x, columns = columns)
    columns = columns[1:]

    for i, ans in enumerate(answers_list):    
        #get list of categories that exist in answer post
        existing_columns = [col for col in columns if col in ans]
        missing_columns = [col for col in columns if col not in ans]

        #fill all missing features in post with UNKNOWN
        answers_df.loc[i, missing_columns] = 'UNKNOWN'         
        
        n = len(existing_columns)
        c = iter(existing_columns)
        curr_c = next(c)

        for j in range(n):          
            #Fill first element column which is 'Answer'   
            try:
                next_c = next(c)
            except StopIteration:
                pass
                        
            if j==0:
                split_post = ans.split(curr_c)
                ans_text = split_post[0]
                answers_df['Answer'].iloc[i] = ans_text
                split_post = split_post[1] 
               
                
            #numeric categories have Count in their names        
            if "Count" in curr_c:
                #in case the category is
                if curr_c!=existing_columns[-1]:
                    #get text relevant to c -column
                    split_post = split_post.split(curr_c)
                    inter_split = split_post[-1].split(next_c)
                    #if category is numeric get numbe
                    answers_df[curr_c].iloc[i] = str_first_num(inter_split[0])
                    split_post = inter_split[-1]
                    curr_c = next_c
                    continue 
                else:
                    answers_df[curr_c].iloc[i] = str_first_num(split_post)
                    continue
        
            #insert regex helper function here----!
            split_post = strip_rgx_words(split_post)
            answers_df[curr_c].iloc[i] = split_post 
            curr_c = next_c 
    return answers_df 

In [20]:
#show one example
answers_into_df(url_list[1])

C:\Users\Ravit\AppData\Local\Temp\ipykernel_32528\789551367.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached


In [ ]:
columns = ["question", "Answer", "upvoteCount", "answerCount", "followerCount", "name"]
df = pd.DataFrame(data = [], columns = columns)

for i, url in enumerate(url_list):
    temp_df = answers_into_df(url)
    temp_df["question"] = url
    #rearrange columns order to match main df columns order
    temp_df = temp_df[columns]

    df = df.append(temp_df)


In [ ]:
#copy of function before making changes

def answers_into_df_copy(url):
    '''input: url for scrapping answers to a particular question.
        out: df with answers for each url/question'''
    #request url html
    
    #create soup object with htmal parser
    page_html = get_html_from_url(url)
    soup = BeautifulSoup(page_html)

    answers = soup.find_all('p', {'class':"q-text qu-display--block qu-wordBreak--break-word qu-textAlign--start"})
    #convert soup object to string
    answers = [str(a) for a in answers][0]
    #split answers with text separator
    answers_list = answers.split('"text":')
    #first element is just the header of a page. remove it
    answers_list = answers_list[1:]
    
    #create df with nans to fill it later with values
    columns = ["Answer", "upvoteCount", "answerCount", "followerCount", "name"]
    #columns = ["Answer", "upvoteCount"]
    x_shape = (len(answers_list), len(columns))
    x = np.tile(np.nan, x_shape)
   
    answers_df = pd.DataFrame(x, columns = columns)
    columns = columns[1:]

    for i, ans in enumerate(answers_list):    
        #get list of categories that exist in answer post
        existing_columns = [col for col in columns if col in ans]
        missing_columns = [col for col in columns if col not in ans]

        #fill all missing features in post with UNKNOWN
        answers_df.loc[i, missing_columns] = 'UNKNOWN'         
        
        n = len(existing_columns)
        c = iter(existing_columns)
        curr_c = next(c)

        for j in range(n):          
            #Fill first element column which is 'Answer'   
            try:
                next_c = next(c)
            except StopIteration:
                pass
                        
            if j==0:
                split_post = ans.split(curr_c)
                ans_text = split_post[0]
                answers_df['Answer'].iloc[i] = ans_text
                split_post = split_post[1] 
               
                
            #numeric categories have Count in their names        
            if "Count" in curr_c:
                #in case the category is
                if curr_c!=existing_columns[-1]:
                    #get text relevant to c -column
                    split_post = split_post.split(curr_c)
                    inter_split = split_post[-1].split(next_c)
                    #if category is numeric get numbe
                    answers_df[curr_c].iloc[i] = str_first_num(inter_split[0])
                    split_post = inter_split[-1]
                    curr_c = next_c
                    continue 
                else:
                    answers_df[curr_c].iloc[i] = str_first_num(split_post)
                    continue
        
            #insert regex helper function here----!
            split_post = strip_rgx_words(split_post)
            answers_df[curr_c].iloc[i] = split_post 
            curr_c = next_c 
    return answers_df 

In [ ]:
df

In [ ]:
#save df to csv
save_path = r"C:\Users\Ravit\Documents\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info"
final_path = os.path.join(save_path, "ET_RCS_22SEP15_quora_data.csv")
df.to_csv(final_path)

In [ ]:
#TODO -  scrape all pages from this urls,
        #scrape all groups related to technology trends
     
         